# An API test using an end point

# 1- Importing key modules

In [1]:
import pickle
import pandas as pd 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfTransformer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hassan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;#]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z +_]')
STOPWORDS = stopwords.words('english')
#STOPWORDS.extend(['http','https','www']) # extend stopwords. If We  had few www or https.
STOPWORDS = set(STOPWORDS)

def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join([word for word in text.split() if word not in STOPWORDS]) # delete stopwords from text
    text = text.strip()
    return text

# 2-Load Model & transformer CV

In [3]:
filename = 'savedmodel/linearsvc_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
filename2 = 'savedmodel/tfidf.sav'
tfidf = pickle.load(open(filename2, 'rb'))

# 3-Out of Sample data

In [5]:
headline=["US jobs growth last month hit by weather:Fed President Charles Plosser"]

In [6]:
data=pd.DataFrame(headline,columns=["news"])
data['clean']=[text_prepare(x) for x in data['news']]

In [7]:
vect = pd.DataFrame(tfidf.transform(data.clean).toarray())
my_prediction = loaded_model.predict(vect)
my_prediction

array(['b'], dtype=object)

In [8]:
score=loaded_model._predict_proba_lr(vect)
score

array([[0.46180912, 0.17402825, 0.15805371, 0.20610892]])

In [9]:
prob=score.max(axis=1)
prob

array([0.46180912])

In [11]:
result=headline
result=pd.DataFrame(result,columns=["text"])
result["news-type"]=my_prediction
#result["news-type"]=result['news-type'].map({1 : "b", 2 : "t", 3 :"e", 4:"m"})
result["probability"]=prob
result

,text,news-type,probability
0,US jobs growth last month hit by weather:Fed P...,b,0.461809


In [12]:
json_table = result.to_json(orient='records')  
json_table

'[{"text":"US jobs growth last month hit by weather:Fed President Charles Plosser","news-type":"b","probability":0.4618091222}]'

### 2nd example

In [19]:
headline2=["YouTube updates to include crowd-funding options"]

In [20]:
data=pd.DataFrame(headline2,columns=["news"])
data['clean']=[text_prepare(x) for x in data['news']]

In [21]:
vect = pd.DataFrame(tfidf.transform(data.clean).toarray())
my_prediction = loaded_model.predict(vect)
score=loaded_model._predict_proba_lr(vect)
prob=score.max(axis=1)

In [22]:
result=headline2
result=pd.DataFrame(result,columns=["text"])
result["news-type"]=my_prediction
#result["news-type"]=result['news-type'].map({1 : "b", 2 : "t", 3 :"e", 4:"m"})
result["probability"]=prob

In [23]:
result

,text,news-type,probability
0,YouTube updates to include crowd-funding options,e,0.304271


In [24]:
json_table = result.to_json(orient='records')  
json_table

'[{"text":"YouTube updates to include crowd-funding options","news-type":"e","probability":0.304271435}]'